# VIIRS Reduction

In [34]:
# imports
import os
import xarray
import numpy as np

# Load up a data file

In [4]:
data_path = '/home/xavier/Projects/Oceanography/data/SST/VIIRS'

In [5]:
data_file = os.path.join(data_path, '20130410235000-OSPO-L2P_GHRSST-SSTsubskin-VIIRS_NPP-ACSPO_V2.61-v02.0-fv01.0.nc')

In [10]:
ds = xarray.open_dataset(data_file)

In [11]:
ds

<xarray.Dataset>
Dimensions:                       (time: 1, nj: 5392, ni: 3200)
Coordinates:
  * time                          (time) datetime64[ns] 2013-04-10T23:50:01
    lat                           (nj, ni) float32 ...
    lon                           (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Data variables: (12/14)
    sst_dtime                     (time, nj, ni) timedelta64[ns] ...
    dt_analysis                   (time, nj, ni) float32 ...
    satellite_zenith_angle        (time, nj, ni) float32 ...
    sea_surface_temperature       (time, nj, ni) float32 ...
    sses_bias                     (time, nj, ni) float32 ...
    sses_standard_deviation       (time, nj, ni) float32 ...
    ...                            ...
    quality_level                 (time, nj, ni) float32 ...
    wind_speed                    (time, nj, ni) float32 ...
    brightness_temperature_11um   (time, nj, ni) float32 ...
    brightness_temperature_12um   (time, nj, ni) float32 ...
    brightness_temperature_4um    (time, nj, ni) float32 ...
    brightness_temperature_08um6  (time, nj, ni) float32 ...
Attributes: (12/60)
    geospatial_bounds:                        POLYGON((   8.854  14.228,   12...
    geospatial_first_scanline_first_fov_lat:  48.722836
    geospatial_first_scanline_first_fov_lon:  12.348704
    geospatial_first_scanline_last_fov_lat:   42.716045
    geospatial_first_scanline_last_fov_lon:   51.49002
    geospatial_last_scanline_first_fov_lat:   14.227749
    ...                                       ...
    time_coverage_end:                        20130410T235959Z
    time_coverage_start:                      20130410T235001Z
    title:                                    VIIRS L2P SST
    uuid:                                     2c5d9460-a20d-11e9-bc8c-246e965...
    westernmost_longitude:                    8.853747
    netcdf_version_id:                        4.5.0 of Oct 31 2017 11:38:27 $

# Write a subset

## Grab em

In [50]:
variables = list(ds.keys())
variables

['sst_dtime',
 'dt_analysis',
 'satellite_zenith_angle',
 'sea_surface_temperature',
 'sses_bias',
 'sses_standard_deviation',
 'sea_ice_fraction',
 'l2p_flags',
 'quality_level',
 'wind_speed',
 'brightness_temperature_11um',
 'brightness_temperature_12um',
 'brightness_temperature_4um',
 'brightness_temperature_08um6']

## Pop out the good ones

In [51]:
#keep_vars = ['wind_speed', 'l2p_flags', 'quality_level', 'sea_surface_temperature', 'dt_analysis']
keep_vars = ['l2p_flags', 'quality_level', 'sea_surface_temperature']

In [52]:
for var in keep_vars:
    variables.remove(var)

In [53]:
variables

['sst_dtime',
 'dt_analysis',
 'satellite_zenith_angle',
 'sses_bias',
 'sses_standard_deviation',
 'sea_ice_fraction',
 'wind_speed',
 'brightness_temperature_11um',
 'brightness_temperature_12um',
 'brightness_temperature_4um',
 'brightness_temperature_08um6']

## Strip me

In [54]:
ds_stripped = ds.drop_vars(variables)

In [55]:
ds_stripped 

<xarray.Dataset>
Dimensions:                  (time: 1, nj: 5392, ni: 3200)
Coordinates:
  * time                     (time) datetime64[ns] 2013-04-10T23:50:01
    lat                      (nj, ni) float32 ...
    lon                      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Data variables:
    sea_surface_temperature  (time, nj, ni) float32 ...
    l2p_flags                (time, nj, ni) int16 ...
    quality_level            (time, nj, ni) float32 1.0 1.0 1.0 ... 1.0 1.0 1.0
Attributes: (12/60)
    geospatial_bounds:                        POLYGON((   8.854  14.228,   12...
    geospatial_first_scanline_first_fov_lat:  48.722836
    geospatial_first_scanline_first_fov_lon:  12.348704
    geospatial_first_scanline_last_fov_lat:   42.716045
    geospatial_first_scanline_last_fov_lon:   51.49002
    geospatial_last_scanline_first_fov_lat:   14.227749
    ...                                       ...
    time_coverage_end:                        20130410T235959Z
    time_coverage_start:                      20130410T235001Z
    title:                                    VIIRS L2P SST
    uuid:                                     2c5d9460-a20d-11e9-bc8c-246e965...
    westernmost_longitude:                    8.853747
    netcdf_version_id:                        4.5.0 of Oct 31 2017 11:38:27 $

## Write

In [28]:
ds_stripped.to_netcdf('tmp.nc')  # 60M

## Encode

In [58]:
ds.sea_surface_temperature

<xarray.DataArray 'sea_surface_temperature' (time: 1, nj: 5392, ni: 3200)>
[17254400 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2013-04-10T23:50:01
    lat      (nj, ni) float32 ...
    lon      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Attributes:
    comment:        SST obtained by regression with buoy measurements, sensit...
    long_name:      sea_surface_subskin_temperature
    source:         NOAA
    standard_name:  sea_surface_subskin_temperature
    units:          kelvin
    valid_max:      32767
    valid_min:      -32767

In [30]:
encoding={}
encoding['sea_surface_temperature'] = {'dtype': 'int16', 'scale_factor': 1e-3,
                     'add_offset': 10., 'zlib': True,
                     '_FillValue': -32768,
                     'missing_value': -32768}

## Write again

In [31]:
ds_stripped.to_netcdf('tmp2.nc', encoding=encoding)  # 60M

In [56]:
ds_stripped.to_netcdf('tmp4.nc', encoding=encoding)  # 60M

In [33]:
ds.quality_level[:]

<xarray.DataArray 'quality_level' (time: 1, nj: 5392, ni: 3200)>
[17254400 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2013-04-10T23:50:01
    lat      (nj, ni) float32 ...
    lon      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Attributes:
    comment:        SST quality levels: 5 corresponds to clear-sky pixels and...
    flag_meanings:  missing invalid not_used not_used not_used clear
    flag_values:    [0 1 2 3 4 5]
    long_name:      quality level of SST pixel
    valid_max:      5
    valid_min:      0

In [35]:
np.unique(ds.quality_level)

array([1., 5.], dtype=float32)

## Recast quality_level

In [37]:
ql = ds.quality_level[:].astype(np.int16)
ql

<xarray.DataArray 'quality_level' (time: 1, nj: 5392, ni: 3200)>
array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]], dtype=int16)
Coordinates:
  * time     (time) datetime64[ns] 2013-04-10T23:50:01
    lat      (nj, ni) float32 ...
    lon      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Attributes:
    comment:        SST quality levels: 5 corresponds to clear-sky pixels and...
    flag_meanings:  missing invalid not_used not_used not_used clear
    flag_values:    [0 1 2 3 4 5]
    long_name:      quality level of SST pixel
    valid_max:      5
    valid_min:      0

In [39]:
ds_stripped = ds_stripped.drop_vars('quality_level')

In [40]:
ds_stripped['quality_level'] = ql

In [41]:
ds_stripped

<xarray.Dataset>
Dimensions:                  (time: 1, nj: 5392, ni: 3200)
Coordinates:
  * time                     (time) datetime64[ns] 2013-04-10T23:50:01
    lat                      (nj, ni) float32 ...
    lon                      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Data variables:
    dt_analysis              (time, nj, ni) float32 ...
    sea_surface_temperature  (time, nj, ni) float32 ...
    l2p_flags                (time, nj, ni) int16 ...
    wind_speed               (time, nj, ni) float32 ...
    quality_level            (time, nj, ni) int16 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
Attributes: (12/60)
    geospatial_bounds:                        POLYGON((   8.854  14.228,   12...
    geospatial_first_scanline_first_fov_lat:  48.722836
    geospatial_first_scanline_first_fov_lon:  12.348704
    geospatial_first_scanline_last_fov_lat:   42.716045
    geospatial_first_scanline_last_fov_lon:   51.49002
    geospatial_last_scanline_first_fov_lat:   14.227749
    ...                                       ...
    time_coverage_end:                        20130410T235959Z
    time_coverage_start:                      20130410T235001Z
    title:                                    VIIRS L2P SST
    uuid:                                     2c5d9460-a20d-11e9-bc8c-246e965...
    westernmost_longitude:                    8.853747
    netcdf_version_id:                        4.5.0 of Oct 31 2017 11:38:27 $

In [42]:
ds_stripped.to_netcdf('tmp3.nc', encoding=encoding)  # 94M!!

In [45]:
np.min(ds_stripped.dt_analysis.data)

nan

In [46]:
isf = np.isfinite(ds_stripped.dt_analysis.data)

In [48]:
np.min(ds_stripped.dt_analysis.data[isf])

-12.7

# Fully strip

In [65]:
variables = list(ds.keys())

In [66]:
variables.remove('l2p_flags')

In [64]:
variables

In [67]:
ds_full = ds.drop_vars(variables)

In [68]:
ds_full.to_netcdf('tmp5.nc')

In [69]:
ds_full

<xarray.Dataset>
Dimensions:    (time: 1, nj: 5392, ni: 3200)
Coordinates:
  * time       (time) datetime64[ns] 2013-04-10T23:50:01
    lat        (nj, ni) float32 ...
    lon        (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Data variables:
    l2p_flags  (time, nj, ni) int16 ...
Attributes: (12/60)
    geospatial_bounds:                        POLYGON((   8.854  14.228,   12...
    geospatial_first_scanline_first_fov_lat:  48.722836
    geospatial_first_scanline_first_fov_lon:  12.348704
    geospatial_first_scanline_last_fov_lat:   42.716045
    geospatial_first_scanline_last_fov_lon:   51.49002
    geospatial_last_scanline_first_fov_lat:   14.227749
    ...                                       ...
    time_coverage_end:                        20130410T235959Z
    time_coverage_start:                      20130410T235001Z
    title:                                    VIIRS L2P SST
    uuid:                                     2c5d9460-a20d-11e9-bc8c-246e965...
    westernmost_longitude:                    8.853747
    netcdf_version_id:                        4.5.0 of Oct 31 2017 11:38:27 $

In [72]:
ds_full.lat.data

array([[48.722836 , 48.723488 , 48.72413  , ..., 42.729317 , 42.722687 ,
        42.716045 ],
       [48.716427 , 48.71708  , 48.71772  , ..., 42.72354  , 42.71691  ,
        42.71027  ],
       [48.710022 , 48.710674 , 48.711315 , ..., 42.717762 , 42.711136 ,
        42.70449  ],
       ...,
       [14.2404585, 14.238953 , 14.237454 , ...,  9.943064 ,  9.940037 ,
         9.936999 ],
       [14.234104 , 14.232598 , 14.231099 , ...,  9.936835 ,  9.933808 ,
         9.930772 ],
       [14.227749 , 14.226244 , 14.224744 , ...,  9.930607 ,  9.92758  ,
         9.924543 ]], dtype=float32)

----

In [57]:
tmp4 = xarray.open_dataset('tmp4.nc')
tmp4

<xarray.Dataset>
Dimensions:                  (time: 1, nj: 5392, ni: 3200)
Coordinates:
  * time                     (time) datetime64[ns] 2013-04-10T23:50:01
    lat                      (nj, ni) float32 ...
    lon                      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Data variables:
    sea_surface_temperature  (time, nj, ni) float32 ...
    l2p_flags                (time, nj, ni) int16 ...
    quality_level            (time, nj, ni) float32 ...
Attributes: (12/60)
    geospatial_bounds:                        POLYGON((   8.854  14.228,   12...
    geospatial_first_scanline_first_fov_lat:  48.722836
    geospatial_first_scanline_first_fov_lon:  12.348704
    geospatial_first_scanline_last_fov_lat:   42.716045
    geospatial_first_scanline_last_fov_lon:   51.49002
    geospatial_last_scanline_first_fov_lat:   14.227749
    ...                                       ...
    time_coverage_end:                        20130410T235959Z
    time_coverage_start:                      20130410T235001Z
    title:                                    VIIRS L2P SST
    uuid:                                     2c5d9460-a20d-11e9-bc8c-246e965...
    westernmost_longitude:                    8.853747
    netcdf_version_id:                        4.5.0 of Oct 31 2017 11:38:27 $

In [59]:
tmp4.sea_surface_temperature

<xarray.DataArray 'sea_surface_temperature' (time: 1, nj: 5392, ni: 3200)>
[17254400 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2013-04-10T23:50:01
    lat      (nj, ni) float32 ...
    lon      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Attributes:
    comment:        SST obtained by regression with buoy measurements, sensit...
    long_name:      sea_surface_subskin_temperature
    source:         NOAA
    standard_name:  sea_surface_subskin_temperature
    units:          kelvin
    valid_max:      32767
    valid_min:      -32767

In [ ]:
# Fully 